# Analysis across sessions


In [ ]:
import pandas as pd
import os
import json
import numpy as np
from itertools import groupby
import itertools
import matplotlib.pyplot as plt
from scipy import stats,signal
import matplotlib as mpl
from sklearn.linear_model import LogisticRegression
import random
import re
import csv
from IPython.display import HTML, display, Image
import tabulate
import math as m
import warnings
warnings.filterwarnings('ignore')
import statsmodels.api as sm
import statsmodels.formula.api as smf

mpl.rcParams['lines.linewidth'] = 3
mpl.rcParams['axes.titlesize'] = 18
mpl.rcParams['axes.labelsize'] = 18
mpl.rcParams['lines.markersize'] = 10
mpl.rcParams['xtick.labelsize'] = 20
mpl.rcParams['ytick.labelsize'] = 20
mpl.rcParams['axes.linewidth'] = 3
#mpl.rcParams['xtick.major.size'] = 20
mpl.rcParams['xtick.major.width'] = 4
#mpl.rcParams['xtick.minor.size'] = 10
mpl.rcParams['xtick.minor.width'] = 2
mpl.rcParams['ytick.major.width'] = 4
mpl.rcParams['ytick.minor.width'] = 2

fday = [1,2,3,4,5,6,7,8,9,10]
fsession = [1,2]

In [ ]:
current_path = os.path.abspath(os.getcwd())
parent_path = os.path.abspath(os.path.join(current_path, os.pardir))
grand_parent_path = os.path.abspath(os.path.join(parent_path, os.pardir))
main_path = os.path.abspath(os.path.join(grand_parent_path, os.pardir))

path_results = main_path+'/results/dots/'
path_plots = main_path+'/scripts/analysis/figures/'

In [ ]:
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, main_path+'/scr')
import my_functions as myf

In [ ]:
adf = pd.read_csv(path_results+'preanalyzed.csv')  

userids = adf['userID'].unique()
userids = sorted(userids)
nsub = len(userids)
sessionids = adf['sessionID_x'].unique()
sessionids = sorted(sessionids)

# morning df
mdf = adf[adf['sessionID_x']%2==1]
# evening df
edf = adf[adf['sessionID_x']%2==0]
edf_sin_nan = edf.copy()
edf_sin_nan['sleep']=list(mdf['sleep'])

key_SR = ['mood','food', 'real_stress','sleep']

In [ ]:
list(adf.columns)

In [ ]:
adf[['no_sigma','no_bias','Dsigma_oo','HmiddleDO','sessionID_x','userID']]

In [ ]:
key_PV = ['no_sigma','no_bias','Dsigma_oo','HmiddleDO','Ssigma_oo','HmiddleSO']

In [ ]:
pair_SR_PV,pair_SR_PV2plot = [],[]
for pv in key_PV:
    aux = []
    for sr in key_SR:
        pair_SR_PV.append((sr,pv))
        aux.append((sr,pv))
    pair_SR_PV2plot.append(aux)

In [ ]:
corr_SR_PV, p_SR_PV, SIG_SR_PV, NOS_SR_PV = {},{},{},{}
for pair in pair_SR_PV:
    if 'sleep' in pair:  
        LR = [myf.Linear_Regr(np.array(mdf[mdf['userID']==part][pair[0]]),\
                            np.array(mdf[mdf['userID']==part][pair[1]])) for part in userids]
    else:
        LR = [myf.Linear_Regr(np.array(adf[adf['userID']==part][pair[0]]),\
                            np.array(adf[adf['userID']==part][pair[1]])) for part in userids]
    corr_SR_PV[pair] = [LR[k].r_value for k in range(len(userids))]
    p_SR_PV[pair] = stats.ttest_1samp(corr_SR_PV[pair],0)[1]
    
    LR_p_value = [LR[k].p_value for k in range(len(userids))]
    LR_p_value = np.array(LR_p_value)
    indSIG = np.where(LR_p_value<0.05)
    indNOS = np.where(LR_p_value>=0.05)
    SIG_SR_PV[pair] = [corr_SR_PV[pair][j] for j in indSIG[0]]
    NOS_SR_PV[pair] = [corr_SR_PV[pair][j] for j in indNOS[0]]

In [ ]:
round_p_PV = np.array([myf.roundP(p_SR_PV[(pair)]) for pair in pair_SR_PV])
round_p_PV = np.reshape(round_p_PV,(6,4))
p_value = [['p_value']+key_SR]
ind = -1
for key in key_PV:
    ind += 1
    p_value.append([key]+list(round_p_PV[ind]))

display(HTML(tabulate.tabulate(p_value, tablefmt='html')))

In [ ]:
fig, ax = plt.subplots(6,4,figsize=(22,24))
plt.subplots_adjust(hspace = 1)  

for ii in range(6):
    for jj in range(4):
        ax[ii,jj].hist([SIG_SR_PV[pair_SR_PV2plot[ii][jj]],NOS_SR_PV[pair_SR_PV2plot[ii][jj]]],bins=np.arange(-1,1,0.1),\
                        alpha=0.7,histtype='bar', stacked=True, \
                        color=['gray',[0.95,0.95,0.95]], edgecolor='gray', linewidth=2) 
        if round_p_PV[ii][jj]<0.05:
            ax[ii,jj].text(-0.95,3.5,'ttest p: '+str(round_p_PV[ii][jj]),\
                           ha='left', wrap=True,fontsize=18,weight='bold',color='r')
        else:
            ax[ii,jj].text(-0.95,3.5,'ttest p: '+str(round_p_PV[ii][jj]),\
                           ha='left', wrap=True,fontsize=18)
        ax[ii,jj].text(-0.95,5.5,'mean r: '+str(myf.roundP(np.nanmean(corr_SR_PV[pair_SR_PV2plot[ii][jj]]))), ha='left', wrap=True,fontsize=18)
        ax[ii,jj].set_title(pair_SR_PV2plot[ii][jj][0])
        ax[ii,jj].set_xlim(-1.1,1.1)
        ax[ii,jj].set_ylim(0,11)
        ax[ii,jj].axvline(0,color='k')
        
        
        if ii==5:
            ax[ii,jj].set_xlabel('Pearson corr coeff')
        else:
            ax[ii,jj].axes.get_xaxis().set_visible(False)
        
        if jj==0:
            ax[ii,0].set_ylabel(pair_SR_PV2plot[ii][jj][1])
        else:
            ax[ii,jj].axes.get_yaxis().set_visible(False)
 
plt.tight_layout()
plt.savefig('corr_psychometric_fit_results_SR.png')
plt.show()

In [ ]:
stats.binom_test(4, n=27, p=0.01, alternative='greater')

In [ ]:
p,p1 = [],[]
for x in np.arange(1,28):
    p.append(stats.binom_test(x, n=27, p=0.05, alternative='greater'))
    p1.append(stats.binom_test(x, n=27, p=0.01, alternative='greater'))
plt.plot(np.arange(1,28),p)
plt.plot(np.arange(1,28),p1)
plt.xlabel('number of subjects sig')
plt.ylabel('p-value')
#plt.legend(('p=0.95','p=0.99'))
plt.tight_layout()
plt.savefig('binomial_test.png')
plt.show()

In [ ]:
adf['OKubj_RT_no']*adf['medianRT']

In [ ]:
Mmedian_toffset,Mmax_toffset,Mmax_under100_toffset = [np.zeros((len(userids),len(sessionids))) for _ in range(3)]
ind1 = -1
for part in userids:
    ind1 += 1
    ind2 = -1
    for ses in sessionids:
        ind2 += 1
        if adf[(adf['userID']==part) & (adf['sessionID_x']==ses)].median_toffset.isnull().values.any():
            Mmedian_toffset[ind1,ind2] = np.nan
            Mmax_toffset[ind1,ind2] = np.nan
        else:
            Mmedian_toffset[ind1,ind2] = float(adf[(adf['userID']==part) & (adf['sessionID_x']==ses)].median_toffset)-300
            Mmax_toffset[ind1,ind2] = float(adf[(adf['userID']==part) & (adf['sessionID_x']==ses)].max_toffset)-300
        if Mmax_toffset[ind1,ind2] < 100:
            Mmax_under100_toffset[ind1,ind2] = Mmax_toffset[ind1,ind2]
        else:
            Mmax_under100_toffset[ind1,ind2] = np.nan

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

fig,ax = plt.subplots(ncols=3,figsize=(20,18))

img1 = ax[0].imshow(Mmedian_toffset,cmap="inferno")
divider = make_axes_locatable(ax[0])
cax = divider.append_axes('right', size='5%', pad=0.05)
fig.colorbar(img1, cax=cax, orientation='vertical')
ax[0].set_yticks(np.arange(len(userids)))
ax[0].set_yticklabels(userids)
ax[0].set_ylabel('subject ID')
ax[0].set_xlabel('session number')
ax[0].set_title('median time offset - 300 ms')

img2 = ax[1].imshow(Mmax_toffset,cmap="inferno")
divider = make_axes_locatable(ax[1])
cax = divider.append_axes('right', size='5%', pad=0.05)
fig.colorbar(img2, cax=cax, orientation='vertical')
ax[1].set_yticks(np.arange(len(userids)))
ax[1].set_yticklabels(userids)
ax[1].set_xlabel('session number')
ax[1].set_title('max time offset - 300 ms')

img3 = ax[2].imshow(Mmax_under100_toffset,cmap="inferno")
divider = make_axes_locatable(ax[2])
cax = divider.append_axes('right', size='5%', pad=0.05)
fig.colorbar(img3, cax=cax, orientation='vertical')
ax[2].set_yticks(np.arange(len(userids)))
ax[2].set_yticklabels(userids)
ax[2].set_xlabel('session number')
ax[2].set_title('max time offset (up to 100) - 300 ms')

plt.tight_layout()
plt.savefig(path_plots+'t_offsetNT.png')
plt.show()

## Mean and sem across participants

In [ ]:
adf.columns

In [ ]:
key_name = ['mood','food', 'real_stress','sleep','Dsubj_optout_oo','Ssubj_optout_oo','Dsubj_perf_oo',\
           'Ssubj_perf_oo','subj_perf_no','OKubj_RT_no','Dsubj_RT_oo','Ssubj_RT_oo']

key_name2plot = [key_name[:int(len(key_name)/2)],key_name[int(len(key_name)/2):]]

ap, se = {},{}
for (var, op) in [(ap, np.nanmean), (se, myf.sem)]:
    for key in key_name:
        if key != 'sleep':
            var[key] = [op(np.array(adf[adf['sessionID_x']==sesid][key])) for sesid in sessionids]
        else:
            var[key] = [op(np.array(mdf[mdf['sessionID_x']==sesid][key])) for sesid in sessionids[0::2]]
session_labels = ['Th','Fr','Sa','Su','Mo','Tu','We','Th','Fr','Sa']

In [ ]:
fig, ax = plt.subplots(6,2,figsize=(18,18))
plt.subplots_adjust(hspace = 0.3)  

for i in range(6):
    for j in range(2):
        lower_limit = np.min(ap[key_name2plot[j][i]])-np.max(se[key_name2plot[j][i]])
        upper_limit = np.max(ap[key_name2plot[j][i]])+np.max(se[key_name2plot[j][i]])
        ax[i,j].fill_between([4,8],lower_limit,upper_limit,color=[0.9,0.9,0.9])
        ax[i,j].fill_between([18,20],lower_limit,upper_limit,color=[0.9,0.9,0.9])
        ax[i,j].set_xticks(np.arange(1,21)[::2])
        ax[i,j].set_xticklabels(session_labels,rotation=45)
        ax[i,j].set_ylabel(key_name2plot[j][i])
        if key_name2plot[j][i]!='sleep':
            ax[i,j].plot(np.arange(1,len(ap[key_name2plot[j][i]])+1), ap[key_name2plot[j][i]], color = 'b')
            ax[i,j].fill_between(np.arange(1,len(ap[key_name2plot[j][i]])+1),np.array(ap[key_name2plot[j][i]])-se[key_name2plot[j][i]], \
                       np.array(ap[key_name2plot[j][i]])+se[key_name2plot[j][i]], alpha=0.2)   
        else:
            ax[i,j].plot(np.arange(1,21)[::2],ap[key_name2plot[j][i]],color='b')
            ax[i,j].fill_between(np.arange(1,21)[::2],\
                       np.array(ap[key_name2plot[j][i]])-se[key_name2plot[j][i]], \
                       np.array(ap[key_name2plot[j][i]])+se[key_name2plot[j][i]], alpha=0.2)  
        
          

plt.show()

In [ ]:
labels = ['Th M','Th A','Fr M','Fr A','Sa M','SaA','Su M','Su A','Mo M','Mo A','Tu M','Tu A',\
          'We M','We A','Th M','Th A','Fr M','Fr A','Sa M','SaA']

fig, ax = plt.subplots(5,2,figsize=(18,18))
plt.subplots_adjust(hspace = 0.5)  

for i in range(5):
    for j in range(2):
        if not (i==0 and j ==1):
            ax[i,j].set_xticks(np.arange(1,len(ap['mood'])+1))
            ax[i,j].set_xticklabels(labels, rotation = 45, ha="right")
            
ax[0,0].set_xticklabels(labels)
ax[0,0].errorbar(np.arange(1,len(ap['mood'])+1), ap['mood'], yerr=se['mood'], fmt='o', color = 'b')
ax[0,0].set_ylabel('Mood report')

ax[0,1].set_xticks(np.arange(1,(len(ap['mood'])+1))[::2])
ax[0,1].set_xticklabels(labels[::2], rotation = 45, ha="right")
ax[0,1].set_xticklabels(labels[::2])
ax[0,1].errorbar(np.arange(1,len(ap['mood'])+1), ap['sleep'], yerr=se['sleep'], fmt='o', color = 'y')
ax[0,1].set_ylabel('Sleep report')

ax[1,0].set_xticklabels(labels)
ax[1,0].errorbar(np.arange(1,len(ap['mood'])+1), ap['food'], yerr=se['food'], fmt='o', color = 'r')
ax[1,0].set_ylabel('Food report')

ax[1,1].set_xticklabels(labels)
ax[1,1].errorbar(np.arange(1,len(ap['mood'])+1), ap['real_stress'], yerr=se['real_stress'], fmt='o', color = 'g')
ax[1,1].set_ylabel('Stress report')

ax[2,0].set_xticklabels(labels)
ax[2,0].errorbar(np.arange(1,len(ap['mood'])+1), ap['Dsubj_optout_oo'], yerr=se['Dsubj_optout_oo'], fmt='o', color = 'r')
ax[2,0].set_ylabel('Deterministic optout')

ax[2,1].set_xticklabels(labels)
ax[2,1].errorbar(np.arange(1,len(ap['mood'])+1), ap['Ssubj_optout_oo'], yerr=se['Ssubj_optout_oo'], fmt='o', color = 'b')
ax[2,1].set_ylabel('Stochastic optout')

ax[3,0].set_xticklabels(labels)
ax[3,0].errorbar(np.arange(1,len(ap['mood'])+1), ap['Dsubj_perf_oo'], yerr=se['Dsubj_perf_oo'], fmt='o', color = 'r')
ax[3,0].set_ylabel('Deterministic perf')
ax[3,0].set_ylim(70,90)

ax[3,1].set_xticklabels(labels)
ax[3,1].errorbar(np.arange(1,len(ap['mood'])+1), ap['Ssubj_perf_oo'], yerr=se['Ssubj_perf_oo'], fmt='o', color = 'b')
ax[3,1].set_ylabel('Stochastic perf')
ax[3,1].set_xlabel('Session label')
ax[3,1].set_ylim(70,90)

ax[4,0].set_xticklabels(labels)
ax[4,0].errorbar(np.arange(1,len(ap['mood'])+1),ap['subj_perf_no'], yerr=se['subj_perf_no'], fmt='o', color = 'g')
ax[4,0].set_ylabel('NO performance')
ax[4,0].set_xlabel('Session label')
ax[4,0].set_ylim(70,90)

ax[4,1].set_xticklabels(labels)
ax[4,1].errorbar(np.arange(1,len(ap['mood'])+1), ap['OKubj_RT_no'], yerr=se['OKubj_RT_no'], fmt='o', color = 'g')
ax[4,1].set_ylabel('NO RT')
ax[4,1].set_xlabel('Session label')

#plt.savefig('across_sessions.png')

plt.show()

In [ ]:
### DO NOT RUN AGAIN !!!

# write the result in file
filename_ap=path_results+'mean_across_participants.json'
filename_se=path_results+'se_across_participants.json'
# Serializing json  
json_object_ap = json.dumps(ap) 
json_object_se = json.dumps(se)

# Writing to sample.json 
with open(filename_ap, "w") as outfile: 
    outfile.write(json_object_ap) 
with open(filename_se, "w") as outfile: 
    outfile.write(json_object_se) 

## Autocorrelation self-reports

In [ ]:
pvals = []
ind = -1
for part in userids:
    ind += 1
    subset = adf[adf['userID']==part]
    mood = np.array(subset['mood'])
    stress = np.array(subset['real_stress'])
    mask = ~np.isnan(mood) & ~np.isnan(stress)
    slope, intercept, r_value, p_value, stderr = stats.linregress(stress[mask],mood[mask])
    if part==3053:
        print(list(mood))
        print(list(stress))
        print(slope, intercept, r_value, p_value, stderr)
    plt.figure()
    plt.scatter(stress,mood)
    plt.ylabel('mood')
    plt.xlabel('stress') 
    plt.plot(np.arange(-0.1,1.1,0.1),intercept+slope*np.arange(-0.1,1.1,0.1))
    plt.fill_between(np.arange(-0.1,1.1,0.1),intercept+slope*np.arange(-0.1,1.1,0.1)-stderr, \
                       intercept+slope*np.arange(-0.1,1.1,0.1)+stderr, color='b', alpha=0.2)
    plt.yticks([0.0,0.5,1.0])
    plt.xticks([0.0,0.5,1.0])
    plt.title(str(int(part))+'  '+str(p_value))
    pvals.append(p_value)
print(np.min(pvals))

## Relationship between reports

In [ ]:
corr_mo_fo,corr_mo_st,corr_st_fo = [[] for _ in range(3)]
corr_sl_mo, corr_sl_fo, corr_sl_st = [[] for _ in range(3)]

ind = -1
for part in userids:
    ind += 1
    subset = adf[adf['userID']==part]
    mood = np.array(subset['mood'])
    stress = np.array(subset['real_stress'])
    food = np.array(subset['food'])
    
    corr_mo_fo.append(myf.Pearson_corr_coef(mood,food))
    corr_mo_st.append(myf.Pearson_corr_coef(mood,stress))
    corr_st_fo.append(myf.Pearson_corr_coef(stress,food))
    
    #morning
    subset_sl = mdf[mdf['userID']==part]
    sleep = np.array(subset_sl['sleep'])
    mood_mor = np.array(subset_sl['mood'])
    food_mor = np.array(subset_sl['food'])
    stress_mor = np.array(subset_sl['real_stress'])

    corr_sl_mo.append(myf.Pearson_corr_coef(sleep,mood_mor))
    corr_sl_fo.append(myf.Pearson_corr_coef(sleep,food_mor))
    corr_sl_st.append(myf.Pearson_corr_coef(sleep,stress_mor))

p_mo_fo = stats.ttest_1samp(corr_mo_fo,0)[1]
p_mo_st = stats.ttest_1samp(corr_mo_st,0)[1]
p_st_fo = stats.ttest_1samp(corr_st_fo,0)[1]
p_sl_mo = stats.ttest_1samp(corr_sl_mo,0)[1]
p_sl_fo = stats.ttest_1samp(corr_sl_fo,0)[1]
p_sl_st = stats.ttest_1samp(corr_sl_st,0)[1]

ptable = [['','mood','food','stress','sleep'],
          ['mood',0,myf.roundP(p_mo_fo),myf.roundP(p_mo_st),myf.roundP(p_sl_mo)],
         ['food',myf.roundP(p_mo_fo),0,myf.roundP(p_st_fo),myf.roundP(p_sl_fo)],
         ['stress',myf.roundP(p_mo_st),myf.roundP(p_st_fo),0,myf.roundP(p_sl_st)],
         ['sleep',myf.roundP(p_sl_mo),myf.roundP(p_sl_fo),myf.roundP(p_sl_st),0]]

display(HTML(tabulate.tabulate(ptable, tablefmt='html')))

In [ ]:
fig,ax = plt.subplots(1,3,figsize=(15,3))
ax[0].hist(corr_mo_st,color='r')
ax[0].set_title("mood & stress")
ax[0].set_xlabel('Pearson correlation coef',fontsize=18)
ax[0].set_ylabel('Counts',fontsize=18)
ax[0].text(0.05,3,'ttest p: '+str(myf.roundP(p_mo_st)), ha='left', wrap=True,fontsize=18)
ax[0].text(0.05,2.5,'mean p: '+str(myf.roundP(np.nanmean(corr_mo_st))), ha='left', wrap=True,fontsize=18)
ax[0].set_xlim(-1.1,1.1)
ax[0].axvline(0,color='k')

ax[1].hist(corr_mo_fo,color='y')
ax[1].set_title("mood & food")
ax[1].set_xlabel('Pearson correlation coef',fontsize=18)
ax[1].text(0.05,3,'ttest p: '+str(myf.roundP(p_mo_fo)), ha='left', wrap=True,fontsize=18)
ax[1].text(0.05,2.5,'mean p: '+str(myf.roundP(np.nanmean(corr_mo_fo))), ha='left', wrap=True,fontsize=18)
ax[1].set_xlim(-1.1,1.1)
ax[1].axvline(0,color='k')

ax[2].hist(corr_st_fo,color='g')
ax[2].set_title("stress & food")
ax[2].set_xlabel('Pearson correlation coef',fontsize=18)
ax[2].text(0.05,3,'ttest p: '+str(myf.roundP(p_st_fo)), ha='left', wrap=True,fontsize=18)
ax[2].text(0.05,2.5,'mean p: '+str(myf.roundP(np.nanmean(corr_st_fo))), ha='left', wrap=True,fontsize=18)
ax[2].set_xlim(-1.1,1.1)
ax[2].axvline(0,color='k')

plt.show()

In [ ]:
fig,ax = plt.subplots(1,3,figsize=(15,3))
ax[0].hist(corr_sl_mo,color='r')
ax[0].set_title("mood & sleep")
ax[0].set_xlabel('Pearson correlation coef',fontsize=18)
ax[0].set_ylabel('Counts',fontsize=18)
ax[0].text(0.05,3,'ttest p: '+str(myf.roundP(p_sl_mo)), ha='left', wrap=True,fontsize=18)
ax[0].text(0.05,2.5,'mean p: '+str(myf.roundP(np.nanmean(corr_sl_mo))), ha='left', wrap=True,fontsize=18)
ax[0].set_xlim(-1.1,1.1)
ax[0].axvline(0,color='k')

ax[1].hist(corr_sl_fo,color='y')
ax[1].set_title("sleep & food")
ax[1].set_xlabel('Pearson correlation coef',fontsize=18)
ax[1].text(0.05,3,'ttest p: '+str(myf.roundP(p_sl_fo)), ha='left', wrap=True,fontsize=18)
ax[1].text(0.05,2.5,'mean p: '+str(myf.roundP(np.nanmean(corr_sl_fo))), ha='left', wrap=True,fontsize=18)
ax[1].set_xlim(-1.1,1.1)
ax[1].axvline(0,color='k')

ax[2].hist(corr_sl_st,color='g')
ax[2].set_title("stress & sleep")
ax[2].set_xlabel('Pearson correlation coef',fontsize=18)
ax[2].text(0.05,3,'ttest p: '+str(myf.roundP(p_sl_st)), ha='left', wrap=True,fontsize=18)
ax[2].text(0.05,2.5,'mean p: '+str(myf.roundP(np.nanmean(corr_sl_st))), ha='left', wrap=True,fontsize=18)
ax[2].set_xlim(-1.1,1.1)
ax[2].axvline(0,color='k')

plt.show()

In [ ]:
matrixR = [[1,np.nanmean(corr_mo_st),np.nanmean(corr_sl_mo),np.nanmean(corr_mo_fo)],\
           [np.nanmean(corr_mo_st),1,np.nanmean(corr_sl_st),np.nanmean(corr_st_fo)],\
           [np.nanmean(corr_sl_mo),np.nanmean(corr_sl_st),1,np.nanmean(corr_sl_fo)],\
           [np.nanmean(corr_mo_fo),np.nanmean(corr_st_fo),np.nanmean(corr_sl_fo),1]]

In [ ]:
print(matrixR)

In [ ]:
pmatrix = [[0,p_mo_st,p_sl_mo,p_mo_fo],
         [p_mo_st,0,p_sl_st,p_st_fo],
         [p_sl_mo,p_sl_st,0,p_sl_fo],
         [p_mo_fo,p_st_fo,p_sl_fo,0]]

In [ ]:
print(pmatrix)

In [ ]:
plt.imshow(matrixR,cmap='seismic',interpolation='nearest', vmin=-1, vmax=1)
plt.xticks(np.arange(4),[])
plt.yticks(np.arange(4),['mood','stress','sleep','food'])
plt.colorbar()
plt.title('Numerosity task')
#plt.tight_layout()
#plt.savefig(path_plots+'reports_corr_dots.png')
plt.show()

In [ ]:
r_subjNOS_fo_mo,r_subjNOS_st_mo,r_subjNOS_sl_mo = [[] for _ in range(3)]
r_subjSIG_fo_mo,r_subjSIG_st_mo,r_subjSIG_sl_mo = [[] for _ in range(3)]
ind = -1
for part in userids:
    ind += 1
    subset = adf[adf['userID']==part]
    mood = np.array(subset['mood'])
    stress = np.array(subset['real_stress'])
    food = np.array(subset['food'])
        
    # food & mood
    LR_fo = myf.Linear_Regr(food,mood)
    if LR_fo.p_value<0.05:
        r_subjSIG_fo_mo.append(LR_fo.r_value)
    else:
        r_subjNOS_fo_mo.append(LR_fo.r_value)
        
    # stress & mood
    LR_st = myf.Linear_Regr(stress,mood)
    if LR_st.p_value<0.05:
        r_subjSIG_st_mo.append(LR_st.r_value)
    else:
        r_subjNOS_st_mo.append(LR_st.r_value)
        
    # morning
    subset_mor = mdf[mdf['userID']==part]
    sleep = np.array(subset_mor['sleep'])
    mood_mor = np.array(subset_mor['mood'])
    
    # sleep & mood
    LR_sl = myf.Linear_Regr(sleep,mood_mor)
    if LR_sl.p_value<0.05:
        r_subjSIG_sl_mo.append(LR_sl.r_value)
    else:
        r_subjNOS_sl_mo.append(LR_sl.r_value)

In [ ]:
# write the result in file
dict_ = {
    "r_subjSIG_st_mo":list(r_subjSIG_st_mo),
    "r_subjNOS_st_mo":list(r_subjNOS_st_mo),
    "r_subjSIG_fo_mo":list(r_subjSIG_fo_mo),
    "r_subjNOS_fo_mo":list(r_subjNOS_fo_mo),
    "r_subjSIG_sl_mo":list(r_subjSIG_sl_mo),
    "r_subjNOS_sl_mo":list(r_subjNOS_sl_mo),
}
# Serializing json  
json_object = json.dumps(dict_) 

# Writing to sample.json 
with open(path_results+'corr_reports_reports.json', "w") as outfile: 
    outfile.write(json_object) 

## Correlation between performance and reports

In [ ]:
DAcc_corr,SAcc_corr,NOAcc_corr = {},{},{}
for key in key_SR:
    if key != 'sleep':
        DAcc_corr[key] = [myf.Pearson_corr_coef(np.array(adf[adf['userID']==part][key]),\
                                        np.array(adf[adf['userID']==part]['Dsubj_perf_oo'])) for part in userids]
        SAcc_corr[key] = [myf.Pearson_corr_coef(np.array(adf[adf['userID']==part][key]),\
                                        np.array(adf[adf['userID']==part]['Ssubj_perf_oo'])) for part in userids]
        NOAcc_corr[key] = [myf.Pearson_corr_coef(np.array(adf[adf['userID']==part][key]),\
                                        np.array(adf[adf['userID']==part]['subj_perf_no'])) for part in userids]
    else:
        DAcc_corr[key] = [myf.Pearson_corr_coef(np.array(mdf[mdf['userID']==part][key]),\
                                        np.array(mdf[mdf['userID']==part]['Dsubj_perf_oo'])) for part in userids]
        SAcc_corr[key] = [myf.Pearson_corr_coef(np.array(mdf[mdf['userID']==part][key]),\
                                        np.array(mdf[mdf['userID']==part]['Ssubj_perf_oo'])) for part in userids]
        NOAcc_corr[key] = [myf.Pearson_corr_coef(np.array(mdf[mdf['userID']==part][key]),\
                                       np.array(mdf[mdf['userID']==part]['subj_perf_no'])) for part in userids]

DAcc_p,SAcc_p,NOAcc_p = {},{},{}                                         
for key in key_SR:        
    DAcc_p[key] = stats.ttest_1samp(DAcc_corr[key],0)[1]  
    SAcc_p[key] = stats.ttest_1samp(SAcc_corr[key],0)[1] 
    NOAcc_p[key] = stats.ttest_1samp(NOAcc_corr[key],0)[1] 
    
ptable = [['p_value']+key_SR,['DAcc']+[myf.roundP(DAcc_p[key]) for key in key_SR],\
          ['SAcc']+[myf.roundP(SAcc_p[key]) for key in key_SR],\
         ['NOAcc']+[myf.roundP(NOAcc_p[key]) for key in key_SR]]

display(HTML(tabulate.tabulate(ptable, tablefmt='html')))

In [ ]:
plt.hist(DAcc_corr['real_stress'],color='r')
plt.xlim(-1,1)
plt.axvline(0,color='k')
plt.xlabel('Pearson corr')
plt.ylabel('Counts')
plt.title('D perf & stress')
plt.text(0.2,2.5,'p-value: '+str(myf.roundP(DAcc_p['real_stress'])), ha='left', wrap=True,fontsize=18)
plt.text(0.2,1.5,'mean r: '+str(myf.roundP(np.nanmean(DAcc_corr['real_stress']))), ha='left', wrap=True,fontsize=18)
plt.show()

## Correlation between optout and reports

In [ ]:
D_subj_corr,S_subj_corr = {},{}
for key in key_SR:
    if key != 'sleep':
        D_subj_corr[key] = [myf.Linear_Regr(np.array(adf[adf['userID']==part][key]),\
                                        np.array(adf[adf['userID']==part]['Dsubj_optout_oo'])) for part in userids]
        S_subj_corr[key] = [myf.Linear_Regr(np.array(adf[adf['userID']==part][key]),\
                                        np.array(adf[adf['userID']==part]['Ssubj_optout_oo'])) for part in userids]
    else:
        D_subj_corr[key] = [myf.Linear_Regr(np.array(mdf[mdf['userID']==part][key]),\
                                        np.array(mdf[mdf['userID']==part]['Dsubj_optout_oo'])) for part in userids]
        S_subj_corr[key] = [myf.Linear_Regr(np.array(mdf[mdf['userID']==part][key]),\
                                        np.array(mdf[mdf['userID']==part]['Ssubj_optout_oo'])) for part in userids]
D_subj_p,S_subj_p = {},{}                                      
for key in key_SR:        
    D_subj_p[key] = [D_subj_corr[key][i].p_value for i in range(nsub)]
    S_subj_p[key] = [S_subj_corr[key][i].p_value for i in range(nsub)]
    
r_subjNOS_DO,r_subjSIG_DO,r_subjNOS_SO,r_subjSIG_SO = [{} for _ in range(4)]    
for key in key_SR: 
    r_subjNOS_DO[key] = [D_subj_corr[key][i].r_value for i in range(nsub) if D_subj_p[key][i]>=0.05]
    r_subjSIG_DO[key] = [D_subj_corr[key][i].r_value for i in range(nsub) if D_subj_p[key][i]<0.05]
    r_subjNOS_SO[key] = [S_subj_corr[key][i].r_value for i in range(nsub) if S_subj_p[key][i]>=0.05]
    r_subjSIG_SO[key] = [S_subj_corr[key][i].r_value for i in range(nsub) if S_subj_p[key][i]<0.05]

In [ ]:
# Serializing json  
json_object_NOS_DO = json.dumps(r_subjNOS_DO) 
json_object_SIG_DO = json.dumps(r_subjSIG_DO) 
json_object_NOS_SO = json.dumps(r_subjNOS_SO) 
json_object_SIG_SO = json.dumps(r_subjSIG_SO) 

# Writing to sample.json 
with open(path_results+'corr_NOS_reports_DO.json', "w") as outfile: 
    outfile.write(json_object_NOS_DO) 
with open(path_results+'corr_SIG_reports_DO.json', "w") as outfile: 
    outfile.write(json_object_SIG_DO) 
with open(path_results+'corr_NOS_reports_SO.json', "w") as outfile: 
    outfile.write(json_object_NOS_SO) 
with open(path_results+'corr_SIG_reports_SO.json', "w") as outfile: 
    outfile.write(json_object_SIG_SO) 

In [ ]:
fig ,(ax1,ax2) = plt.subplots(2,1,gridspec_kw={'hspace':0})
ax1.set_title("optout & mood")
ax1.hist([r_subjNOS_DO['mood'],r_subjSIG_DO['mood']],bins=np.arange(-1,1,0.1),alpha=0.7,histtype='bar', stacked=True, \
             color=['w','r'], edgecolor='r', linewidth=2)
ax1.set_yticks([0,3,6])
ax1.set_xlim(-1.1,1.1)
ax1.set_ylabel('counts')
ax1.set_xticks([])
ax1.set_ylim(0,8)
ax2.hist([r_subjNOS_SO['mood'],r_subjSIG_SO['mood']],bins=np.arange(-1,1,0.1),color=['w','b'],alpha=0.7,histtype='bar',\
         stacked=True,edgecolor='b', linewidth=2)
ax2.set_yticks([3,6])
ax2.set_xticks([-1,0,1])
ax2.set_xlim(-1.1,1.1)
ax2.set_ylim(0,8)
ax2.invert_yaxis()

## Correlation between response time and reports

In [ ]:
DRT_corr,SRT_corr,DRTOK_corr,SRTOK_corr,NORTOK_corr = [{} for _ in range(5)]
for key in key_SR:
    if key != 'sleep':
        DRTOK_corr[key] = [myf.Pearson_corr_coef(np.array(adf[adf['userID']==part][key]),\
                            np.array(adf[adf['userID']==part]['OKDsubj_RT_oo'])) for part in userids]
        SRTOK_corr[key] = [myf.Pearson_corr_coef(np.array(adf[adf['userID']==part][key]),\
                            np.array(adf[adf['userID']==part]['OKSsubj_RT_oo'])) for part in userids]
        NORTOK_corr[key] = [myf.Pearson_corr_coef(np.array(adf[adf['userID']==part][key]),\
                            np.array(adf[adf['userID']==part]['OKubj_RT_no'])) for part in userids]
    else:
        DRTOK_corr[key] = [myf.Pearson_corr_coef(np.array(mdf[mdf['userID']==part][key]),\
                            np.array(mdf[mdf['userID']==part]['OKDsubj_RT_oo'])) for part in userids]
        SRTOK_corr[key] = [myf.Pearson_corr_coef(np.array(mdf[mdf['userID']==part][key]),\
                            np.array(mdf[mdf['userID']==part]['OKSsubj_RT_oo'])) for part in userids]
        NORTOK_corr[key] = [myf.Pearson_corr_coef(np.array(mdf[mdf['userID']==part][key]),\
                            np.array(mdf[mdf['userID']==part]['OKubj_RT_no'])) for part in userids]

DRTOK_p,SRTOK_p,NORTOK_p = [{} for _ in range(3)]                                         
for key in key_SR:        
    DRTOK_p[key] = stats.ttest_1samp(DRTOK_corr[key],0)[1]  
    SRTOK_p[key] = stats.ttest_1samp(SRTOK_corr[key],0)[1] 
    NORTOK_p[key] = stats.ttest_1samp(NORTOK_corr[key],0)[1] 
    
ptable = [['p_value']+key_SR,['D_RT_OK']+[myf.roundP(DRTOK_p[key]) for key in key_SR],\
          ['S_RT_OK']+[myf.roundP(SRTOK_p[key]) for key in key_SR],\
         ['NO_RT_OK']+[myf.roundP(NORTOK_p[key]) for key in key_SR]]

display(HTML(tabulate.tabulate(ptable, tablefmt='html')))

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(10,3))
ax[0].hist(DRTOK_corr['mood'],color='r')
ax[0].text(-0.95,3.5,'ttest p: '+str(myf.roundP(DRTOK_p['mood'])), ha='left', wrap=True,fontsize=18)
ax[0].text(-0.95,4.5,'mean r: '+str(myf.roundP(np.nanmean(DRTOK_corr['mood']))), ha='left', wrap=True,fontsize=18)
ax[0].set_title("DO OK RT & mood")
ax[0].set_xlabel('Pearson corr coeff')
ax[0].set_ylabel('Counts')
ax[0].set_xlim(-1.1,1.1)
ax[0].axvline(0,color='k')

ax[1].hist(NORTOK_corr['mood'],color='g')
ax[1].text(0.1,2.5,'ttest p: '+str(myf.roundP(NORTOK_p['mood'])), ha='left', wrap=True,fontsize=18)
ax[1].text(0.1,3.5,'mean r: '+str(myf.roundP(np.nanmean(NORTOK_corr['mood']))), ha='left', wrap=True,fontsize=18)
ax[1].set_title("NO OK RT & mood")
ax[1].set_xlabel('Pearson corr coeff')
ax[1].set_ylabel('Counts')
ax[1].set_xlim(-1.1,1.1)
ax[1].axvline(0,color='k')

plt.show()

## Correlation between psychometric variables

In [ ]:
key_PV = ['subj_perf_no','OKubj_RT_no','Dsubj_optout_oo','Ssubj_optout_oo']
pair_keys_PV = list(itertools.combinations(key_PV, 2))

corr_PV = {}
for pair in pair_keys_PV:
    corr_PV[pair] = [myf.Pearson_corr_coef(np.array(adf[adf['userID']==part][pair[0]]),\
                            np.array(adf[adf['userID']==part][pair[1]])) for part in userids]

p_PV = {}                           
for pair in pair_keys_PV:       
    p_PV[pair] = stats.ttest_1samp(corr_PV[pair],0)[1]  
    
M = [myf.roundP(p_PV[(pair)]) for pair in pair_keys_PV]

p_value=[['p_value']+key_PV,[key_PV[0],0]+M[0:3],[key_PV[1],M[0],0]+M[3:5],[key_PV[2],M[1],M[3],0,M[5]],\
         [key_PV[3],M[2],M[4],M[5],0]]

display(HTML(tabulate.tabulate(p_value, tablefmt='html')))

R = [myf.roundP(np.mean(corr_PV[(pair)])) for pair in pair_keys_PV]

mean_r=[['mean_corr']+key_PV,[key_PV[0],0]+R[0:3],[key_PV[1],R[0],0]+R[3:5],[key_PV[2],R[1],R[3],0,R[5]],\
         [key_PV[3],R[2],R[4],R[5],0]]

display(HTML(tabulate.tabulate(mean_r, tablefmt='html')))

In [ ]:
r_subjNOS_so_do,r_subjSIG_so_do = [[] for _ in range(2)]
ind = -1
for part in userids:
    ind += 1
    subset = adf[adf['userID']==part]
    DO = np.array(subset['Dsubj_optout_oo'])
    SO = np.array(subset['Ssubj_optout_oo'])
    
    # D optout & S optout
    slope_so, intercept_so, r_value_so, p_value_so, std_err_so = stats.linregress(SO,DO)
    if p_value_so<0.05:
        r_subjSIG_so_do.append(r_value_so)
    else:
        r_subjNOS_so_do.append(r_value_so)

In [ ]:
# write the result in file
dict_ = {
    "r_subjSIG_so_do":list(r_subjSIG_so_do),
    "r_subjNOS_so_do":list(r_subjNOS_so_do)
}
# Serializing json  
json_object = json.dumps(dict_) 

# Writing to sample.json 
with open(path_results+'corr_Doptout_Soptout.json', "w") as outfile: 
    outfile.write(json_object) 

In [ ]:
fig, ax = plt.subplots()

ax.hist(corr_PV[('Dsubj_optout_oo','Ssubj_optout_oo')],color='m')
ax.text(-0.8,3.5,'ttest p: '+str(myf.roundP(p_PV[('Dsubj_optout_oo','Ssubj_optout_oo')])), ha='left', wrap=True,fontsize=18)
ax.text(-0.8,2.5,'mean r: '+str(myf.roundP(np.nanmean(corr_PV[('Dsubj_optout_oo','Ssubj_optout_oo')]))), ha='left', wrap=True,fontsize=18)
ax.set_title("DO & SO")
ax.set_xlabel('Pearson corr coeff')
ax.set_xlim(-1.1,1.1)
ax.axvline(0,color='k')

plt.show()

## Overconfidence

### Noise and bias with and without optout

We compare noise and bias in both type of trials. To compare bias, in trials with optout, we use the middle point $(H_L+H_R)/2$.

In [ ]:
df3054 = adf[adf['userID']==3054]

In [ ]:
sigma_no = np.array(df3054['no_sigma'])
sigmaDoo = np.array(df3054['Dsigma_oo'])
sigmaSoo = np.array(df3054['Ssigma_oo'])
mean_sigma_no = np.nanmean(sigma_no)
sd_sigma_no = np.nanstd(sigma_no)
mean_sigmaDoo = np.nanmean(sigmaDoo)
sd_sigmaDoo = np.nanstd(sigmaDoo)
mean_sigmaSoo = np.nanmean(sigmaSoo)
sd_sigmaSoo = np.nanstd(sigmaSoo)

sigma_no = np.array(df3054['no_sigma'])
sigmaDoo = np.array(df3054['Dsigma_oo'])
sigmaSoo = np.array(df3054['Ssigma_oo'])

# mask to avoid nan values
mask = ~np.isnan(sigma_no) & ~np.isnan(sigmaDoo)
maskS = ~np.isnan(sigma_no) & ~np.isnan(sigmaSoo)

In [ ]:
bias_no

In [ ]:
slope, intercept, r_value, p_value, std_err = stats.linregress(sigma_no[mask],sigmaDoo[mask])
slopeS, interceptS, r_valueS, p_valueS, std_errS = stats.linregress(sigma_no[maskS],sigmaSoo[maskS])

fig,ax = plt.subplots(1,2,figsize=(16,5))
plt.subplots_adjust(wspace = 0.3)
ax[0].plot()
ax[0].plot(np.arange(-0.08,0.07,0.01),intercept+np.array(np.arange(-0.08,0.07,0.01))*slope,'k')
ax[0].scatter(sigma_no,sigmaDoo,color='r')
ax[0].set_xlabel('sigma non-optout')
ax[0].set_ylabel('sigma deterministic optout')

ax[1].plot()
ax[1].plot(np.arange(-0.08,0.07,0.01),interceptS+np.array(np.arange(-0.08,0.07,0.01))*slopeS,'k')
ax[1].scatter(sigma_no,sigmaSoo,color='r')
ax[1].set_xlabel('sigma non-optout')
ax[1].set_ylabel('sigma stochastic optout')

plt.show()

In [ ]:
resultD = stats.linregress(bias_no[mask],biasDoo[mask])
resultS = stats.linregress(bias_no[maskS],biasSoo[maskS])

In [ ]:
resultD.slope

In [ ]:
sigma_no = np.array(adf['no_sigma'])
sigmaDoo = np.array(adf['Dsigma_oo'])
sigmaSoo = np.array(adf['Ssigma_oo'])
mean_sigma_no = np.nanmean(sigma_no)
sd_sigma_no = np.nanstd(sigma_no)
mean_sigmaDoo = np.nanmean(sigmaDoo)
sd_sigmaDoo = np.nanstd(sigmaDoo)
mean_sigmaSoo = np.nanmean(sigmaSoo)
sd_sigmaSoo = np.nanstd(sigmaSoo)

df_PCfit = adf[(adf['no_sigma']<mean_sigma_no+2*sd_sigma_no) & (adf['no_sigma']>mean_sigma_no-2*sd_sigma_no)]
df_PCfit = adf[(adf['Dsigma_oo']<mean_sigmaDoo+2*sd_sigmaDoo) & (adf['Dsigma_oo']>mean_sigmaDoo-2*sd_sigmaDoo)]
df_PCfit = adf[(adf['Ssigma_oo']<mean_sigmaSoo+2*sd_sigmaSoo) & (adf['Ssigma_oo']>mean_sigmaSoo-2*sd_sigmaSoo)]

sigma_no = np.array(df_PCfit['no_sigma'])
sigmaDoo = np.array(df_PCfit['Dsigma_oo'])
sigmaSoo = np.array(df_PCfit['Ssigma_oo'])

# mask to avoid nan values
mask = ~np.isnan(sigma_no) & ~np.isnan(sigmaDoo)
maskS = ~np.isnan(sigma_no) & ~np.isnan(sigmaSoo)

In [ ]:
slope, intercept, r_value, p_value, std_err = stats.linregress(sigma_no[mask],sigmaDoo[mask])
slopeS, interceptS, r_valueS, p_valueS, std_errS = stats.linregress(sigma_no[maskS],sigmaSoo[maskS])

fig,ax = plt.subplots(1,2,figsize=(16,5))
plt.subplots_adjust(wspace = 0.3)
ax[0].plot()
ax[0].plot(sigmaDoo,intercept+np.array(sigmaDoo)*slope,'k')
ax[0].scatter(sigma_no,sigmaDoo,color='r')
ax[0].set_xlabel('Sigma NON optout')
ax[0].set_ylabel('Sigma deterministic optout')
ax[0].text(0.1,0.1,'ttest p: '+str(myf.roundP(p_value)), ha='left', wrap=True,fontsize=18)
ax[0].text(0.1,0.15,'mean r: '+str(myf.roundP(np.mean(r_value))), ha='left', wrap=True,fontsize=18)
#ax[0].set_xscale('log')
#ax[0].set_yscale('log')

ax[1].plot()
ax[1].plot(sigmaSoo,interceptS+np.array(sigmaSoo)*slopeS,'k')
ax[1].scatter(sigma_no,sigmaSoo,color='r')
ax[1].set_xlabel('Sigma NON optout')
ax[1].set_ylabel('Sigma stochastic optout')
ax[1].text(0.1,0.1,'ttest p: '+str(myf.roundP(p_valueS)), ha='left', wrap=True,fontsize=18)
ax[1].text(0.1,0.15,'mean r: '+str(myf.roundP(np.mean(r_valueS))), ha='left', wrap=True,fontsize=18)
#ax[1].set_xscale('log')
#ax[1].set_yscale('log')
plt.show()

In [ ]:
bias_no = np.array(adf['no_bias'])
HmiddleDO = np.array(adf['HmiddleDO'])
HmiddleSO = np.array(adf['HmiddleSO'])
mean_bias_no = np.nanmean(bias_no)
sd_bias_no = np.nanstd(bias_no)
mean_HmiddleDO = np.nanmean(HmiddleDO)
sd_HmiddleDO = np.nanstd(HmiddleDO)
mean_HmiddleSO = np.nanmean(HmiddleSO)
sd_HmiddleSO = np.nanstd(HmiddleSO)

df_PCfit2 = adf[(adf['no_sigma']<mean_bias_no+2*sd_bias_no) & (adf['no_sigma']>mean_bias_no-2*sd_bias_no)]
df_PCfit2 = adf[(adf['HmiddleDO']<mean_HmiddleDO+2*sd_HmiddleDO) & (adf['HmiddleDO']>mean_HmiddleDO-2*sd_HmiddleDO)]
df_PCfit2 = adf[(adf['HmiddleSO']<mean_HmiddleSO+2*sd_HmiddleSO) & (adf['HmiddleSO']>mean_HmiddleSO-2*sd_HmiddleSO)]

bias_no = np.array(df_PCfit2['no_sigma'])
HmiddleDO = np.array(df_PCfit2['HmiddleDO'])
HmiddleSO = np.array(df_PCfit2['HmiddleSO'])
# mask to avoid nan values
mask = ~np.isnan(bias_no) & ~np.isnan(HmiddleDO)
maskS = ~np.isnan(bias_no) & ~np.isnan(HmiddleSO)

In [ ]:
slope, intercept, r_value, p_value, std_err = stats.linregress(bias_no[mask],HmiddleDO[mask])
slopeS, interceptS, r_valueS, p_valueS, std_errS = stats.linregress(bias_no[maskS],HmiddleSO[maskS])

fig,ax = plt.subplots(1,2,figsize=(17,5))
plt.subplots_adjust(wspace = 0.3)
ax[0].plot()
ax[0].plot(bias_no,intercept+np.array(bias_no)*slope,'k')
ax[0].scatter(bias_no,HmiddleDO)
ax[0].set_xlabel('Bias NON optout')
ax[0].set_ylabel('Bias deterministic optout')
ax[0].text(0.07,0.,'ttest p: '+np.format_float_scientific(myf.roundP(p_value)), ha='left', wrap=True,fontsize=18)
ax[0].text(0.07,0.03,'mean r: '+str(myf.roundP(np.mean(r_value))), ha='left', wrap=True,fontsize=18)

ax[1].plot()
ax[1].plot(bias_no,interceptS+np.array(bias_no)*slopeS,'k')
ax[1].scatter(bias_no,HmiddleSO)
ax[1].set_xlabel('Bias NON optout')
ax[1].set_ylabel('Bias stochastic optout')
ax[1].text(0.07,0.,'ttest p: '+np.format_float_scientific(myf.roundP(p_valueS)), ha='left', wrap=True,fontsize=18)
ax[1].text(0.07,0.03,'mean r: '+str(myf.roundP(np.mean(r_valueS))), ha='left', wrap=True,fontsize=18)
plt.show()

## Overconfidence

Defined as $\frac{|H_L^{opt}-H_R^{opt}| - |H_L-H_R|}{|H_L^{opt}-H_R^{opt}|+|H_L-H_R|}$, we study the correlation between overconfidence and self-reports. We calculate $H_R^{opt}$ for the optout trials, defined as the stimulus value where $p(x>0|\hat{x})=2/3$. The optimal bias for leftward response corresponds to the stimulus value where $p(x<0|\hat{x})=1/3$: $H_L^{opt}=-H_R^{opt}$.

In [ ]:
DoverC = np.array(adf['DoverConf'])
SoverC = np.array(adf['SoverConf'])

fig, ax = plt.subplots(1,2,figsize=(15,4)) 
plt.subplots_adjust(wspace = 0.4) 
ax[0].hist(DoverC,color='r')
ax[0].set_title("Deterministic Overconfidence")
ax[0].set_ylabel('Counts')
ax[0].set_xlim(-1.1,1.1)
ax[0].axvline(color='k')

ax[1].hist(SoverC,color='b')
ax[1].set_title("Stochastic Overconfidence")
ax[1].set_xlim(-1.1,1.1)
ax[1].axvline(color='k')

plt.show()

In [ ]:
Dover_corr,Sover_corr = {},{}
for key in key_SR:
    if key != 'sleep':
        Dover_corr[key] = [myf.Pearson_corr_coef(np.array(adf[adf['userID']==part][key]),\
                                        np.array(adf[adf['userID']==part]['DoverConf'])) for part in userids]
        Sover_corr[key] = [myf.Pearson_corr_coef(np.array(adf[adf['userID']==part][key]),\
                                        np.array(adf[adf['userID']==part]['SoverConf'])) for part in userids]
    else:
        Dover_corr[key] = [myf.Pearson_corr_coef(np.array(mdf[mdf['userID']==part][key]),\
                                        np.array(mdf[mdf['userID']==part]['DoverConf'])) for part in userids]
        Sover_corr[key] = [myf.Pearson_corr_coef(np.array(mdf[mdf['userID']==part][key]),\
                                        np.array(mdf[mdf['userID']==part]['SoverConf'])) for part in userids]

Dover_p,Sover_p = {},{}                                       
for key in key_SR:        
    Dover_p[key] = stats.ttest_1samp(Dover_corr[key],0)[1]  
    Sover_p[key] = stats.ttest_1samp(Sover_corr[key],0)[1] 
    
ptable = [['p_value']+key_SR,['Dover']+[myf.roundP(Dover_p[key]) for key in key_SR],\
          ['Sover']+[myf.roundP(Sover_p[key]) for key in key_SR]]

display(HTML(tabulate.tabulate(ptable, tablefmt='html')))

## Risk aversion

We define the risk aversion as the difference between the choice opf opt-out option in the *DO* stage and the same amount in the *SO*.

In [ ]:
Dcorr_ra_mo,Dcorr_ra_fo,Dcorr_ra_sl,Dcorr_ra_st = [[] for _ in range(4)]
ind = -1
for part in userids:
    ind += 1
    subset = adf[adf['userID']==part]
    mood = np.array(subset['mood'])
    stress = np.array(subset['real_stress'])
    food = np.array(subset['food'])
    RA = np.array(subset['risk_av'])
    if not m.isnan(np.corrcoef(mood,RA)[0][1]):
        Dcorr_ra_mo.append(np.corrcoef(mood,RA)[0][1])
    if not m.isnan(np.corrcoef(stress,RA)[0][1]):
        Dcorr_ra_st.append(np.corrcoef(stress,RA)[0][1])
    if not m.isnan(np.corrcoef(food,RA)[0][1]):
        Dcorr_ra_fo.append(np.corrcoef(food,RA)[0][1])
    # morning
    subset_mor = adf[(adf['userID']==part) & (adf['sessionID_x']%2==1)]
    sleep = np.array(subset_mor['sleep'])
    RAmor = subset_mor['risk_av']
    if not m.isnan(np.corrcoef(sleep,RAmor)[0][1]):
        Dcorr_ra_sl.append(np.corrcoef(sleep,RAmor)[0][1])

Dp_ra_mo = stats.ttest_1samp(Dcorr_ra_mo,0)[1]    
Dp_ra_fo = stats.ttest_1samp(Dcorr_ra_fo,0)[1]  
Dp_ra_sl = stats.ttest_1samp(Dcorr_ra_sl,0)[1]  
Dp_ra_st = stats.ttest_1samp(Dcorr_ra_st,0)[1]     

ptable = [['','Risk Aversion'],
          ['mood',np.round(Dp_ra_mo,2)],
         ['food',np.round(Dp_ra_fo,2)],
         ['stress',np.round(Dp_ra_st,2)],
         ['sleep',np.round(Dp_ra_sl,2)]]

#display(Latex(r"\newpage"))

display(HTML(tabulate.tabulate(ptable, tablefmt='html')))

In [ ]:
# delete rows with nan values of discarded data
adf_sin_nan = adf.drop([349,444])

## LMM optout & reports

In [ ]:
mm_oo_re_RE = smf.mixedlm("Dsubj_optout_oo ~ mood + stress + sleep + food",Rdf_sin_nan,groups=Rdf_sin_nan["userID"],\
                         re_formula="~mood + stress + sleep + food")
mdf_oo_re_RE = mm_oo_re_RE.fit()
print(mdf_oo_re_RE.summary())

In [ ]:
mm_oo_FE_FE = smf.mixedlm("Dsubj_optout_oo ~ mood + stress + sleep + food", adf_sin_nan, groups=adf_sin_nan["userID"])
mdf_oo_FE_FE = mm_oo_FE_FE.fit()
print(mdf_oo_FE_FE.summary())

In [ ]:
#  DeltaAIC = 2LLH(sinfixedeffect) +2 - 2LLH(full)
DeltaAIC = 2*(-2163.3747) +2 - 2*(-2131.0432)

In [ ]:
print(DeltaAIC)

In [ ]:
adf.columns

In [ ]:
bins = np.arange(0.1,1,0.1)
meanDoptout,sdDoptout,meanSoptout,sdSoptout,leng = [[] for _ in range(5)]
for i in range(10):
    bi = 0.1*(i+1)
    subset = adf[(adf['mood']<bi) & (adf['mood']>(bi-0.1))]
    DO = np.array(subset['Dsubj_optout_oo'])
    SO = np.array(subset['Ssubj_optout_oo'])
    meanDoptout.append(np.nanmean(DO))
    sdDoptout.append(np.nanstd(DO))
    meanSoptout.append(np.nanmean(SO))
    sdSoptout.append(np.nanstd(SO))
    leng.append(len(subset))

In [ ]:
width = 0.05
plt.bar(x=np.arange(0.,1.,0.1)+width/2, #x-coordinates of bars
       height=meanDoptout, #height of bars
       yerr=sdDoptout/np.sqrt(leng), #error bar width
       capsize=4,width=width,color='r',alpha=0.7) #length of error bar caps
plt.bar(x=np.arange(0.,1.,0.1)+width*3/2, #x-coordinates of bars
       height=meanSoptout, #height of bars
       yerr=sdSoptout/np.sqrt(leng), #error bar width
       capsize=4,width=width,color='b',alpha=0.7) #length of error bar caps
plt.ylim(10,55)
plt.xlabel('mood')
plt.ylabel('optout (%)')
plt.legend(("Deterministic", "Stochastic"),loc='upper left',shadow=True)
plt.tight_layout()
#plt.savefig('BARoptoutVSstress_all_data_together.png')
plt.show()

In [ ]:
meanDoptout,sdDoptout,meanSoptout,sdSoptout,leng = [[] for _ in range(5)]
for i in range(10):
    bi = 0.1*(i+1)
    subset = adf[(adf['real_stress']<bi) & (adf['real_stress']>(bi-0.1))]
    DO = np.array(subset['Dsubj_optout_oo'])
    SO = np.array(subset['Ssubj_optout_oo'])
    meanDoptout.append(np.nanmean(DO))
    sdDoptout.append(np.nanstd(DO))
    meanSoptout.append(np.nanmean(SO))
    sdSoptout.append(np.nanstd(SO))
    leng.append(len(subset))

In [ ]:
plt.plot(bins,meanDoptout,color='r')
plt.fill_between(np.arange(0.,1.,0.1),meanDoptout-sdDoptout/np.sqrt(leng),meanDoptout+sdDoptout/np.sqrt(leng),color='r', alpha=0.2)

plt.plot(bins,meanSoptout,color='b')
plt.fill_between(np.arange(0.,1.,0.1),meanSoptout-sdSoptout/np.sqrt(leng),meanSoptout+sdSoptout/np.sqrt(leng),color='b', alpha=0.2)

plt.xlabel('stress')
plt.ylabel('optout (%)')
plt.legend(("Deterministic", "Stochastic"),loc='upper left',shadow=True)

plt.tight_layout()
plt.savefig('optoutVSstress_all_data_together.png')
plt.show()

In [ ]:
width = 0.05
plt.bar(x=np.arange(0.,1.,0.1)+width/2, #x-coordinates of bars
       height=meanDoptout, #height of bars
       yerr=sdDoptout/np.sqrt(leng), #error bar width
       capsize=4,width=width,color='r',alpha=0.7) #length of error bar caps
plt.bar(x=np.arange(0.,1.,0.1)+width*3/2, #x-coordinates of bars
       height=meanSoptout, #height of bars
       yerr=sdSoptout/np.sqrt(leng), #error bar width
       capsize=4,width=width,color='b',alpha=0.7) #length of error bar caps
plt.ylim(16,55)
plt.xlabel('stress')
plt.ylabel('optout (%)')
plt.legend(("Deterministic", "Stochastic"),loc='upper left',shadow=True)
plt.tight_layout()
plt.savefig('BARoptoutVSstress_all_data_together.png')
plt.show()

In [ ]:
meanDoptout,sdDoptout,meanSoptout,sdSoptout,leng = [[] for _ in range(5)]
for i in range(5):
    bi = 0.2*(i+1)
    subset = adf[(adf['real_stress']<bi) & (adf['real_stress']>(bi-0.2))]
    DO = np.array(subset['Dsubj_optout_oo'])
    SO = np.array(subset['Ssubj_optout_oo'])
    meanDoptout.append(np.nanmean(DO))
    sdDoptout.append(np.nanstd(DO))
    meanSoptout.append(np.nanmean(SO))
    sdSoptout.append(np.nanstd(SO))
    leng.append(len(subset))

In [ ]:
width = 0.05
plt.bar(x=np.arange(0.,1.,0.2)+width/2, #x-coordinates of bars
       height=meanDoptout, #height of bars
       yerr=sdDoptout/np.sqrt(leng), #error bar width
       capsize=4,width=width,color='r',alpha=0.7) #length of error bar caps
plt.bar(x=np.arange(0.,1.,0.2)+width*3/2, #x-coordinates of bars
       height=meanSoptout, #height of bars
       yerr=sdSoptout/np.sqrt(leng), #error bar width
       capsize=4,width=width,color='b',alpha=0.7) #length of error bar caps
plt.ylim(16,55)
plt.xlabel('stress')
plt.ylabel('optout (%)')
plt.legend(("Deterministic", "Stochastic"),loc='upper left',shadow=True)
plt.tight_layout()
plt.savefig('BAR5bins_optoutVSstress_all_data_together.png')
plt.show()

In [ ]:
mm_oo_stress_FE = smf.mixedlm("Dsubj_optout_oo ~ stress", Rdf_sin_nan, groups=Rdf_sin_nan["userID"])
mdf_oo_stress_FE = mm_oo_stress_FE.fit()
print(mdf_oo_stress_FE.summary())

In [ ]:
mm_oo_stress_RE = smf.mixedlm("Dsubj_optout_oo ~ stress", Rdf_sin_nan, groups=Rdf_sin_nan["userID"],re_formula="~stress")
mdf_oo_stress_RE = mm_oo_stress_RE.fit()
print(mdf_oo_stress_RE.summary())

In [ ]:
stress_todos = np.array(Rdf_sin_nan['real_stress'])
DO_todos = np.array(Rdf_sin_nan['Dsubj_optout_oo'])
slope, intercept, r_value, p_value, std_err = stats.linregress(stress_todos,DO_todos)
plt.scatter(stress_todos,DO_todos,c='r',alpha=0.7)
plt.plot(np.arange(-0.1,1.1,0.1),intercept+slope*np.arange(-0.1,1.1,0.1),'k')
#plt.fill_between(np.arange(-0.1,1.1,0.1),intercept+slope*np.arange(-0.1,1.1,0.1)-std_err, \
#                   intercept+slope*np.arange(-0.1,1.1,0.1)+std_err, color='b', alpha=0.2)
plt.text(-0.12,87,'p='+str(roundP(p_value)), ha='left', wrap=True,fontsize=18)
plt.xlabel('stress')
plt.ylabel('optout (%)')
plt.tight_layout()
plt.savefig('DoptoutVSstress_all_data_together.png')
plt.show()

In [ ]:
SO_todos = np.array(Rdf_sin_nan['Ssubj_optout_oo'])
slope, intercept, r_value, p_value, std_err = stats.linregress(stress_todos,SO_todos)
plt.scatter(stress_todos,SO_todos,c='b',alpha=0.7)
plt.plot(np.arange(-0.1,1.1,0.1),intercept+slope*np.arange(-0.1,1.1,0.1),'k')
#plt.fill_between(np.arange(-0.1,1.1,0.1),intercept+slope*np.arange(-0.1,1.1,0.1)-std_err, \
#                   intercept+slope*np.arange(-0.1,1.1,0.1)+std_err, color='b', alpha=0.2)
plt.text(-0.12,94,'p='+str(roundP(p_value)), ha='left', wrap=True,fontsize=18)
plt.xlabel('stress')
plt.ylabel('optout (%)')
plt.tight_layout()
plt.savefig('SoptoutVSstress_all_data_together.png')
plt.show()

In [ ]:
bins = np.arange(0.1,1,0.1)
meanDoptout,sdDoptout,meanSoptout,sdSoptout,leng = [[] for _ in range(5)]
for i in range(10):
    bi = 0.1*(i+1)
    subset = adf[(adf['sleep']<bi) & (adf['sleep']>(bi-0.1))]
    DO = np.array(subset['Dsubj_optout_oo'])
    SO = np.array(subset['Ssubj_optout_oo'])
    meanDoptout.append(np.nanmean(DO))
    sdDoptout.append(np.nanstd(DO))
    meanSoptout.append(np.nanmean(SO))
    sdSoptout.append(np.nanstd(SO))
    leng.append(len(subset))

In [ ]:
width = 0.05
plt.bar(x=np.arange(0.,1.,0.1)+width/2, #x-coordinates of bars
       height=meanDoptout, #height of bars
       yerr=sdDoptout/np.sqrt(leng), #error bar width
       capsize=4,width=width,color='r',alpha=0.7) #length of error bar caps
plt.bar(x=np.arange(0.,1.,0.1)+width*3/2, #x-coordinates of bars
       height=meanSoptout, #height of bars
       yerr=sdSoptout/np.sqrt(leng), #error bar width
       capsize=4,width=width,color='b',alpha=0.7) #length of error bar caps
plt.ylim(16,55)
plt.xlabel('sleep')
plt.ylabel('optout (%)')
plt.legend(("Deterministic", "Stochastic"),loc='upper left',shadow=True)
plt.tight_layout()
#plt.savefig('BARoptoutVSstress_all_data_together.png')
plt.show()

In [ ]:
mean_stressAS,mean_DOAS,mean_SOAS = [[] for _ in range(3)]
std_stressAS,std_DOAS,std_SOAS = [[] for _ in range(3)]
for part in userids:
    subset = adf[adf['userID']==part]
    stress = np.array(subset['real_stress'])
    DO = np.array(subset['Dsubj_optout_oo'])
    SO = np.array(subset['Ssubj_optout_oo'])
    # delete nan values
    stress = stress[~np.isnan(stress)]
    DO = DO[~np.isnan(DO)]
    SO = SO[~np.isnan(SO)]

    mean_stressAS.append(np.mean(stress))
    mean_DOAS.append(np.mean(DO))
    mean_SOAS.append(np.mean(SO))
    std_stressAS.append(np.std(stress))
    std_DOAS.append(np.std(DO))
    std_SOAS.append(np.std(SO))

In [ ]:
slope, intercept, r_value, p_value, std_err = stats.linregress(mean_stressAS,mean_DOAS)
plt.scatter(mean_stressAS,mean_DOAS,c='r',alpha=0.7)
plt.plot(np.arange(-0.1,1.1,0.1),intercept+slope*np.arange(-0.1,1.1,0.1),'r')
plt.fill_between(np.arange(-0.1,1.1,0.1),intercept+slope*np.arange(-0.1,1.1,0.1)-std_err, \
                   intercept+slope*np.arange(-0.1,1.1,0.1)+std_err, color='r', alpha=0.2)
plt.text(-0.12,66,'p='+str(roundP(p_value)), ha='left', wrap=True,fontsize=18)
plt.xlabel('mean stress')
plt.ylabel('mean D optout (%)')
plt.tight_layout()
plt.savefig('meanDoptoutVSmeanStress_across_sessions.png')
plt.show()

In [ ]:
slope, intercept, r_value, p_value, std_err = stats.linregress(mean_stressAS,mean_SOAS)
plt.scatter(mean_stressAS,mean_SOAS,c='b',alpha=0.7)
plt.plot(np.arange(-0.1,1.1,0.1),intercept+slope*np.arange(-0.1,1.1,0.1),'b')
plt.fill_between(np.arange(-0.1,1.1,0.1),intercept+slope*np.arange(-0.1,1.1,0.1)-std_err, \
                   intercept+slope*np.arange(-0.1,1.1,0.1)+std_err, color='b', alpha=0.2)
plt.text(-0.12,72,'p='+str(roundP(p_value)), ha='left', wrap=True,fontsize=18)
plt.xlabel('mean stress')
plt.ylabel('mean S optout (%)')
plt.tight_layout()
plt.savefig('meanSoptoutVSmeanStress_across_sessions.png')
plt.show()

In [ ]:
slope, intercept, r_value, p_value, std_err = stats.linregress(mean_stressAS,std_DOAS)
plt.scatter(mean_stressAS,std_DOAS,c='r',alpha=0.7)
plt.plot(np.arange(-0.1,1.1,0.1),intercept+slope*np.arange(-0.1,1.1,0.1),'r')
#plt.fill_between(np.arange(-0.1,1.1,0.1),intercept+slope*np.arange(-0.1,1.1,0.1)-std_err, \
#                   intercept+slope*np.arange(-0.1,1.1,0.1)+std_err, color='b', alpha=0.2)
plt.text(-0.12,26,'p='+str(roundP(p_value)), ha='left', wrap=True,fontsize=18)
plt.xlabel('stress')
plt.ylabel('std D optout (%)')
plt.tight_layout()
#plt.savefig('SoptoutVSstress_all_data_together.png')
plt.show()

In [ ]:
doble_mean_stressAS = np.concatenate((mean_stressAS,mean_stressAS))
double_optout = np.concatenate((mean_DOAS,mean_SOAS))
slope, intercept, r_value, p_value, std_err = stats.linregress(doble_mean_stressAS,double_optout)
plt.scatter(doble_mean_stressAS,double_optout,c='m',alpha=0.7)
plt.plot(np.arange(-0.1,1.1,0.1),intercept+slope*np.arange(-0.1,1.1,0.1),'m')
#plt.fill_between(np.arange(-0.1,1.1,0.1),intercept+slope*np.arange(-0.1,1.1,0.1)-std_err, \
#                   intercept+slope*np.arange(-0.1,1.1,0.1)+std_err, color='b', alpha=0.2)
plt.text(-0.12,70,'p='+str(roundP(p_value)), ha='left', wrap=True,fontsize=18)
plt.xlabel('stress')
plt.ylabel('D+S optout (%)')
plt.tight_layout()
plt.savefig('mean_D+SoptoutVSmean_stress.png')
plt.show()

In [ ]:
print(np.min(std_DOAS),np.max(std_DOAS))

In [ ]:
print(np.min(std_stressAS),np.max(std_stressAS))

In [ ]:
from pingouin import ancova

In [ ]:
mood_todos = np.array(Rdf_sin_nan['mood'])

In [ ]:
optout_todos = np.concatenate((DO_todos,SO_todos))
optout_type = ['D']*len(DO_todos)+['S']*len(SO_todos)
doble_stress = np.concatenate((stress_todos,stress_todos))
doble_mood = np.concatenate((mood_todos,mood_todos))
users = np.concatenate((np.array(Rdf_sin_nan['userID']),np.array(Rdf_sin_nan['userID'])))

In [ ]:
ancova_df = pd.DataFrame({'userID':users,'stress':doble_stress,'optout':optout_todos,'optout_type':optout_type})

In [ ]:
print(ancova(data=ancova_df, dv='optout', covar='stress', between='optout_type'))

In [ ]:
mm_DSoo_stress_FE = smf.mixedlm("optout ~ stress", ancova_df, groups=ancova_df["userID"])
mdf_DSoo_stress_FE = mm_DSoo_stress_FE.fit()
print(mdf_DSoo_stress_FE.summary())

In [ ]:
mm_DSoo_stress_RE = smf.mixedlm("optout ~ stress", ancova_df, groups=ancova_df["userID"],re_formula="~stress")
mdf_DSoo_stress_RE = mm_DSoo_stress_RE.fit()
print(mdf_DSoo_stress_RE.summary())

In [ ]:
adf.columns

In [ ]:
mm_Doo_ID_RE = smf.mixedlm("Dsubj_optout_oo ~ userID", Rdf_sin_nan, groups=Rdf_sin_nan["userID"])
mdf_Doo_ID_RE = mm_Doo_ID_RE.fit()
print(mdf_Doo_ID_RE.summary())

In [ ]:
food_todos = np.array(Rdf_sin_nan['food'])
slope, intercept, r_value, p_value, std_err = stats.linregress(food_todos,DO_todos)
plt.scatter(food_todos,DO_todos,c='r',alpha=0.7)
plt.plot(np.arange(-0.1,1.1,0.1),intercept+slope*np.arange(-0.1,1.1,0.1),'k')
#plt.fill_between(np.arange(-0.1,1.1,0.1),intercept+slope*np.arange(-0.1,1.1,0.1)-std_err, \
#                   intercept+slope*np.arange(-0.1,1.1,0.1)+std_err, color='b', alpha=0.2)
plt.text(-0.12,87,'p='+str(roundP(p_value)), ha='left', wrap=True,fontsize=18)
plt.xlabel('food')
plt.ylabel('optout (%)')
plt.tight_layout()
#plt.savefig('DoptoutVSfood_all_data_together.png')
plt.show()

In [ ]:
corr_oo_st = []
ind = -1
for part in userids:
    ind += 1
    subset = ancova_df[ancova_df['userID']==part]
    stress = np.array(subset['stress'])
    OO = np.array(subset['optout'])
    
    corr_oo_st.append(np.corrcoef(stress,OO)[0][1])

p_oo_st = stats.ttest_1samp(corr_oo_st,0)[1]   

print(p_oo_st)